In [322]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [323]:
%run ../notebook_preamble.ipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [324]:
import geopandas as gpd

In [325]:
import os

In [326]:
from urllib.request import urlretrieve
from zipfile import ZipFile
from beis_indicators import project_dir

In [327]:
from beis_indicators.geo.coders import NutsCoder, LepCoder
# from beis_indicators.geo.nuts import auto_nuts2_uk
from beis_indicators.indicators import points_to_indicator

In [328]:
lsoa_shp_01 = gpd.read_file('../../data/raw/Lower_Layer_Super_Output_Areas__December_2001__EW_BGC-shp/Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.shp')
lsoa_shp_11 = gpd.read_file('../../data/raw/Lower_Layer_Super_Output_Areas__December_2011__Boundaries_EW_BGC_v3-shp/Lower_Layer_Super_Output_Areas__December_2011__Boundaries_EW_BGC_v3.shp')

In [329]:
# LSOA to lon/lat
lsoa_shp_geo_01 = lsoa_shp_01.to_crs(epsg=4326)
lsoa_shp_geo_11 = lsoa_shp_11.to_crs(epsg=4326)

In [330]:
lsoa_shp_geo_11.head(2)

,OBJECTID,LSOA11CD,LSOA11NM,LSOA11NMW,Age_Indica,Shape__Are,Shape__Len,geometry
0,1,E01000001,City of London 001A,City of London 001A,0,133320.768867,2291.846072,POLYGON ((-0.0972886732320328 51.5215770420208...
1,2,E01000002,City of London 001B,City of London 001B,0,226191.273003,2433.960112,POLYGON ((-0.08812915078150488 51.519410719510...


In [331]:
# print(lsoa_shp_geo.iloc[0].geometry.centroid.x, lsoa_shp_geo.iloc[0].geometry.centroid.y)

lsoa_shp_geo_01['lon'] = lsoa_shp_geo_01.geometry.apply(lambda i: i.centroid.x)
lsoa_shp_geo_01['lat'] = lsoa_shp_geo_01.geometry.apply(lambda i: i.centroid.y)

lsoa_shp_geo_11['lon'] = lsoa_shp_geo_11.geometry.apply(lambda i: i.centroid.x)
lsoa_shp_geo_11['lat'] = lsoa_shp_geo_11.geometry.apply(lambda i: i.centroid.y)

In [332]:
lsoa_shp_geo_re_01 = lsoa_shp_geo_01.rename(columns={'LSOA01CD':'LSOA_code'})
lsoa_shp_geo_re_11 = lsoa_shp_geo_11.rename(columns={'LSOA11CD':'LSOA_code'})

2011 data (England and Wales) uses 2001 LSOA data <br />
2013 data (England only) uses 2011 LSOA data

- Read more here: http://data.dft.gov.uk.s3.amazonaws.com/connectivity-data/Brief-guide-to-connectivity-travel-time-data_v2.pdf

## Road Junctions

Method: Averaging time over the nearest 5 road junctions 

Time of day: AM

Mode of transport: Car

In [358]:
#2011 data
df_road = pd.read_csv('../../data/raw/travel/Road-junctions-travel-times/Junctions_HW_AM.csv')
#2013 data
df_road_13 = pd.read_csv('../../data/raw/travel/2013_Junctions_HW_AM.csv')

In [359]:
print(len(df_road))
print(len(df_road_13))

16869696
16051932


In [361]:
df_road['NearOrder']

0

In [272]:
df_road_filter = df_road[df_road['NearOrder'] <= 4]
df_road_filter_13 = df_road_13[df_road_13['NearOrder'] <= 4]

In [273]:
len(df_road_filter)
len(df_road_filter_13)

163770

In [274]:
df_road_filter.reset_index(inplace=True, drop=True)
df_road_filter_13.reset_index(inplace=True, drop=True)

In [275]:
df_road_lat_lon = df_road_filter.merge(lsoa_shp_geo_re_01, on='LSOA_code', how='left')#
df_road_lat_lon_13 = df_road_filter_13.merge(lsoa_shp_geo_re_11, on='LSOA_code', how='left')

In [276]:
df_road_lat_lon = df_road_lat_lon[['LSOA_code', 'RepTime', 'Percentage Services', 'uid', 'NearOrder', 'lon', 'lat']]
df_road_lat_lon_13 = df_road_lat_lon_13[['LSOA_code', 'RepTime', 'Percentage Services', 'UID', 'NearOrder', 'lon', 'lat']]
df_road_lat_lon_13.columns = ['LSOA_code', 'RepTime', 'Percentage Services', 'uid', 'NearOrder', 'lon', 'lat']

In [277]:
df_road_lat_lon['year'] = 2011
df_road_lat_lon_13['year'] = 2013

In [278]:
df_road_latlon_11_13 = pd.concat([df_road_lat_lon, df_road_lat_lon_13]).reset_index(drop=True)

### NUTS 2

In [279]:
df_road_nuts2 = points_to_indicator(df_road_latlon_11_13, value_col='RepTime', coder=NutsCoder(level=2),
                    aggfunc=np.mean, value_rename='travel_to_road_junctions_average_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:43:06,015 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2016 boundaries
2020-10-12 14:43:10,256 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2021 boundaries
2020-10-12 14:43:13,759 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2003 boundaries
2020-10-12 14:43:18,746 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2006 boundaries
2020-10-12 14:43:30,061 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2010 boundaries
2020-10-12 14:43:34,901 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2013 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [280]:
df_road_nuts2 = df_road_nuts2.sort_values(by='nuts_id')

In [281]:
df_road_nuts2.to_csv('../../data/processed/travel/travel_to_road_junctions_average_time.nuts2.csv', index=False)

### NUTS 3

In [282]:
df_road_nuts3 = points_to_indicator(df_road_latlon_11_13, value_col='RepTime', coder=NutsCoder(level=3),
                    aggfunc=np.mean, value_rename='travel_to_road_junctions_average_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:44:21,785 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2016 boundaries
2020-10-12 14:44:27,005 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2021 boundaries
2020-10-12 14:44:31,981 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2003 boundaries
2020-10-12 14:44:39,283 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2006 boundaries
2020-10-12 14:44:55,842 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2010 boundaries
2020-10-12 14:45:04,208 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2013 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [283]:
df_road_nuts3 = df_road_nuts3.sort_values(by='nuts_id')

In [284]:
df_road_nuts3.to_csv('../../data/processed/travel/travel_to_road_junctions_average_time.nuts3.csv', index=False)

### LEP

In [285]:
df_road_lep = points_to_indicator(df_road_latlon_11_13, value_col='RepTime', coder=LepCoder(),
                    aggfunc=np.mean, value_rename='travel_to_road_junctions_average_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:46:05,493 - beis_indicators.geo.coders - INFO - Loading LEP 2014 boundaries
2020-10-12 14:46:18,092 - beis_indicators.geo.coders - INFO - Loading LEP 2017 boundaries
2020-10-12 14:46:27,262 - beis_indicators.geo.coders - INFO - Loading LEP 2020 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [286]:
df_road_lep = df_road_lep.sort_values(by='lep_id')

In [287]:
df_road_lep.to_csv('../../data/processed/travel/travel_to_road_junctions_average_time.lep.csv', index=False)

## Airport

Method: Closest airport 

Time of Day: AM

Mode of transport: Car

In [288]:
df_air_11 = pd.read_csv('../../data/raw/travel/Airports-travel-times/Airports_HW_AM.csv')
df_air_13 = pd.read_csv('../../data/raw/travel/2013_Airports_HW_AM.csv')

In [289]:
df_air_filter_11 = df_air_11[df_air_11['NearOrder'] <= 0]
df_air_filter_13 = df_air_13[df_air_13['NearOrder'] <= 0]

In [290]:
print(len(df_air_filter_11))
print(len(df_air_filter_13))

34288
32754


In [291]:
df_air_filter_11.reset_index(inplace=True, drop=True)
df_air_filter_13.reset_index(inplace=True, drop=True)

In [292]:
df_air_lat_lon = df_air_filter_11.merge(lsoa_shp_geo_re_01, on='LSOA_code', how='left')
df_air_lat_lon_13 = df_air_filter_13.merge(lsoa_shp_geo_re_11, on='LSOA_code', how='left')

In [293]:
df_air_lat_lon = df_air_lat_lon[['LSOA_code', 'RepTime', 'Percentage Services', 'uid', 'NearOrder', 'lon', 'lat']]
df_air_lat_lon_13 = df_air_lat_lon_13[['LSOA_code', 'RepTime', 'Percentage Services', 'UID', 'NearOrder', 'lon', 'lat']]
df_air_lat_lon_13.columns = ['LSOA_code', 'RepTime', 'Percentage Services', 'uid', 'NearOrder', 'lon', 'lat']

In [294]:
df_air_lat_lon['year'] = [2011]*len(df_air_lat_lon)
df_air_lat_lon_13['year'] = [2013]*len(df_air_lat_lon_13)

In [295]:
df_air_latlon_11_13 = pd.concat([df_air_lat_lon, df_air_lat_lon_13]).reset_index(drop=True)

### NUTS 2

In [296]:
df_air_nuts2 = points_to_indicator(df_air_latlon_11_13, value_col='RepTime', coder=NutsCoder(level=2),
                    aggfunc=np.mean, value_rename='travel_to_airport_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:47:41,628 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2016 boundaries
2020-10-12 14:47:45,303 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2021 boundaries
2020-10-12 14:47:48,753 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2003 boundaries
2020-10-12 14:47:53,723 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2006 boundaries
2020-10-12 14:48:04,384 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2010 boundaries
2020-10-12 14:48:09,334 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2013 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [297]:
df_air_nuts2 = df_air_nuts2.sort_values(by='nuts_id')

In [298]:
df_air_nuts2.to_csv('../../data/processed/travel/travel_to_airport_time.nuts2.csv', index=False)

### NUTS 3

In [299]:
df_air_nuts3 = points_to_indicator(df_air_latlon_11_13, value_col='RepTime', coder=NutsCoder(level=3),
                    aggfunc=np.mean, value_rename='travel_to_airport_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:48:21,901 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2016 boundaries
2020-10-12 14:48:27,011 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2021 boundaries
2020-10-12 14:48:31,991 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2003 boundaries
2020-10-12 14:48:39,344 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2006 boundaries
2020-10-12 14:48:55,633 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2010 boundaries
2020-10-12 14:49:03,214 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2013 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [300]:
df_air_nuts3 = df_air_nuts3.sort_values(by='nuts_id')

In [301]:
df_air_nuts3.to_csv('../../data/processed/travel/travel_to_airport_time.nuts3.csv', index=False)

### LEP

In [302]:
df_air_lep = points_to_indicator(df_air_latlon_11_13, value_col='RepTime', coder=LepCoder(),
                    aggfunc=np.mean, value_rename='travel_to_airport_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:49:17,460 - beis_indicators.geo.coders - INFO - Loading LEP 2014 boundaries
2020-10-12 14:49:27,160 - beis_indicators.geo.coders - INFO - Loading LEP 2017 boundaries
2020-10-12 14:49:36,971 - beis_indicators.geo.coders - INFO - Loading LEP 2020 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [303]:
df_air_lep = df_air_lep.sort_values(by='lep_id')

In [304]:
df_air_lep.to_csv('../../data/processed/travel/travel_to_airport_time.lep.csv', index=False)

## Rail Stations

Method: Closest rail station 

Time of Day: AM

Mode of transport: Car

In [305]:
df_rail_11 = pd.read_csv('../../data/raw/travel/Rail-stations-travel-times/Stations_HW_AM.csv')
df_rail_13 = pd.read_csv('../../data/raw/travel/2013_Stations_HW_AM.csv')

In [306]:
df_rail_filter_11 = df_rail_11[df_rail_11['NearOrder'] <= 0]
df_rail_filter_13 = df_rail_13[df_rail_13['NearOrder'] <= 0]

In [307]:
print(len(df_rail_filter_11))
print(len(df_rail_filter_13))

34288
32754


In [308]:
df_rail_filter_11.reset_index(inplace=True, drop=True)
df_rail_filter_13.reset_index(inplace=True, drop=True)

In [309]:
df_rail_lat_lon = df_rail_filter_11.merge(lsoa_shp_geo_re_01, on='LSOA_code', how='left')
df_rail_lat_lon_13 = df_rail_filter_13.merge(lsoa_shp_geo_re_11, on='LSOA_code', how='left')

In [310]:
df_rail_lat_lon = df_rail_lat_lon[['LSOA_code', 'RepTime', 'Percentage Services', 'uid', 'NearOrder', 'lon', 'lat']]
df_rail_lat_lon_13 = df_rail_lat_lon_13[['LSOA_code', 'RepTime', 'Percentage Services', 'UID', 'NearOrder', 'lon', 'lat']]
df_rail_lat_lon_13.columns = ['LSOA_code', 'RepTime', 'Percentage Services', 'uid', 'NearOrder', 'lon', 'lat']

In [311]:
df_rail_lat_lon['year'] = [2011]*len(df_rail_lat_lon)
df_rail_lat_lon_13['year'] = [2013]*len(df_rail_lat_lon_13)

In [312]:
df_rail_latlon_11_13 = pd.concat([df_rail_lat_lon, df_rail_lat_lon_13]).reset_index(drop=True)

### NUTS 2

In [313]:
df_rail_nuts2 = points_to_indicator(df_rail_latlon_11_13, value_col='RepTime', coder=NutsCoder(level=2),
                    aggfunc=np.mean, value_rename='travel_to_rail_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:50:27,739 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2016 boundaries
2020-10-12 14:50:31,424 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2021 boundaries
2020-10-12 14:50:35,029 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2003 boundaries
2020-10-12 14:50:40,524 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2006 boundaries
2020-10-12 14:50:52,498 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2010 boundaries
2020-10-12 14:50:57,515 - beis_indicators.geo.coders - INFO - Loading NUTS 2 2013 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [314]:
df_rail_nuts2 = df_rail_nuts2.sort_values(by='nuts_id')

In [315]:
df_rail_nuts2.to_csv('../../data/processed/travel/travel_to_rail_time.nuts2.csv', index=False)

### NUTS 3

In [316]:
df_rail_nuts3 = points_to_indicator(df_rail_latlon_11_13, value_col='RepTime', coder=NutsCoder(level=3),
                    aggfunc=np.mean, value_rename='travel_to_rail_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:51:16,761 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2016 boundaries
2020-10-12 14:51:25,277 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2021 boundaries
2020-10-12 14:51:31,782 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2003 boundaries
2020-10-12 14:51:40,658 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2006 boundaries
2020-10-12 14:51:58,247 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2010 boundaries
2020-10-12 14:52:06,023 - beis_indicators.geo.coders - INFO - Loading NUTS 3 2013 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [317]:
df_rail_nuts3 = df_rail_nuts3.sort_values(by='nuts_id')

In [318]:
df_rail_nuts3.to_csv('../../data/processed/travel/travel_to_rail_time.nuts3.csv', index=False)

### LEP

In [319]:
df_rail_lep = points_to_indicator(df_air_latlon_11_13, value_col='RepTime', coder=LepCoder(),
                    aggfunc=np.mean, value_rename='travel_to_rail_time',
                    projection='EPSG:4326', x_col='lon', y_col='lat')

2020-10-12 14:52:20,330 - beis_indicators.geo.coders - INFO - Loading LEP 2014 boundaries
2020-10-12 14:52:30,531 - beis_indicators.geo.coders - INFO - Loading LEP 2017 boundaries
2020-10-12 14:52:39,983 - beis_indicators.geo.coders - INFO - Loading LEP 2020 boundaries


/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')
/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/beis_indicators/geo/coders.py:45: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points, shape, op='within', how='right')


In [320]:
df_rail_lep = df_rail_lep.sort_values(by='lep_id')

In [321]:
df_rail_lep.to_csv('../../data/processed/travel/travel_to_rail_time.lep.csv', index=False)

## Travel to Work

In [55]:
xl = pd.ExcelFile('../../data/raw/travel/averagehometoworktraveltimes16plusod18.xls')

In [56]:
xl.sheet_names

['Metadata', 'OD18']

In [57]:
lad_latlon = pd.read_csv('../../data/raw/travel/Local_Authority_Districts__December_2018__Boundaries_UK_BGC.csv')

In [58]:
df = xl.parse('OD18').drop('Office For National Statistics', axis=1)

In [ ]:
#fuzzy search



In [59]:
df.head()

,Unnamed: 1,Unnamed: 2
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [24]:
df.columns = ['UALADGB UA / LAD of residence', 'Mean']


In [25]:
df = df[9:419].reset_index(drop=True)

In [26]:
df['UALADGB UA / LAD of residence'][0]

'  AA City of London'

In [27]:
'47UD Redditch'.split()[1]

'Redditch'

In [28]:
df['LAD'] = df['UALADGB UA / LAD of residence'].apply(lambda x: x.strip().split(" ",1)[1])

In [30]:
del df['UALADGB UA / LAD of residence']

In [33]:
df = df[['LAD', 'Mean']]

In [41]:
len(df)

410

In [37]:
lad_latlon

,objectid,lad18cd,lad18nm,lad18nmw,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,E06000001,Hartlepool,,447157,531476,-1.27023,54.6762,9.364074e+07,69780.416446
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.5447,5.387613e+07,42262.990135
2,3,E06000003,Redcar and Cleveland,,464359,519597,-1.00611,54.5675,2.448759e+08,96548.932570
3,4,E06000004,Stockton-on-Tees,,444937,518183,-1.30669,54.5569,2.049579e+08,115541.332608
4,5,E06000005,Darlington,,428029,515648,-1.56835,54.5353,1.974976e+08,105784.983937
5,6,E06000006,Halton,,354246,382146,-2.68853,53.3342,7.908640e+07,76340.008096
6,7,E06000007,Warrington,,362744,388456,-2.56167,53.3916,1.806453e+08,111798.638796
7,8,E06000008,Blackburn with Darwen,,369490,422806,-2.46360,53.7008,1.370625e+08,63732.321791
8,9,E06000009,Blackpool,,332763,436633,-3.02284,53.8216,3.487525e+07,34580.344897
9,10,E06000010,"Kingston upon Hull, City of",,511894,431716,-0.30380,53.7698,7.146231e+07,63584.402721


In [39]:
lad_latlon = lad_latlon.rename(columns={'lad18nm':'LAD'})


In [45]:
df_merge = df.merge(lad_latlon, on='LAD', how='left')

In [49]:
df_merge[df_merge['objectid'].isna()]

,LAD,Mean,objectid,lad18cd,lad18nmw,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
115,"Anglesey, Isle of",26.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,"Vale of Glamorgan, The",28.3251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,"Rhondda, Cynon, Taff",29.9231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Argyll & Bute,33.2173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,"Scot Borders, The",23.2812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,"Edinburgh, City of",29.1662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,Eilean Siar (Western Isles),34.9226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,Northern Ireland,25.2366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,Abroad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,Mid Bedfordshire,34.9258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Test

In [333]:
url = 'https://opendata.arcgis.com/datasets/180a5c44cfc643c0848813f0a81c1bd1_0.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D'

In [350]:
fname = f'travel_road_junctions_13'

In [335]:
travel_dir = f'{project_dir}/data/raw/travel'
if not os.path.isdir(travel_dir):
    os.mkdir(travel_dir)

In [351]:
fout = f'{travel_dir}/{fname}.zip'

In [352]:
fout

'/mnt/c/Users/aotubusen/Documents/DS Projects/beis_2020/beis-indicators/ds/data/raw/travel/travel_road_junctions_13.zip'

In [338]:
if not os.path.isfile(fout):
    urlretrieve(url, fout)

In [339]:
project_zip_dir = f'{project_dir}/data/raw/travel/shp_01.zip'
# project_zip_dir = BROADBAND_YEARS_URL[year]
# project_zip = ZipFile(project_zip_dir)

In [355]:
project_zip.filename()

TypeError: 'str' object is not callable

In [353]:
with ZipFile(project_zip_dir, 'r') as zip_ref:
    zip_ref.extractall(f'{project_dir}/data/raw/travel/travel_road_junctions_13')

In [357]:
[text_file.filename
               for text_file in project_zip.infolist()
               ]

['Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.shp',
 'Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.dbf',
 'Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.prj',
 'Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.cpg',
 'Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.shx',
 'Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.xml']

In [112]:
project_zip.infolist()

[<ZipInfo filename='Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.shp' compress_type=deflate filemode='-rw-r--r--' file_size=133454512 compress_size=69718529>,
 <ZipInfo filename='Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.dbf' compress_type=deflate filemode='-rw-r--r--' file_size=4434988 compress_size=1300868>,
 <ZipInfo filename='Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.prj' compress_type=deflate filemode='-rw-r--r--' file_size=417 compress_size=281>,
 <ZipInfo filename='Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.cpg' filemode='-rw-r--r--' file_size=5>,
 <ZipInfo filename='Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.shx' compress_type=deflate filemode='-rw-r--r--' file_size=275124 compress_size=178827>,
 <ZipInfo filename='Lower_Layer_Super_Output_Areas__December_2001__EW_BGC.xml' compress_type=deflate filemode='-rw-r--r--' file_size=41165 compress_size=3068>]

In [111]:
project_zip.infolist().filename.endswith('.shp')

AttributeError: 'list' object has no attribute 'filename'

In [348]:
def test():
    return {2001: lsoa_shp_geo_01, 2011: lsoa_shp_geo_11}

In [349]:
test()[2001]

,OBJECTID,LSOA01CD,LSOA01NM,LSOA01NMW,Shape__Are,Shape__Len,geometry,lon,lat
0,1,E01000001,City of London 001A,City of London 001A,1.298432e+05,2370.656583,POLYGON ((-0.09477051357319799 51.520594367473...,-0.096781,51.518036
1,2,E01000002,City of London 001B,City of London 001B,2.278982e+05,2481.826139,POLYGON ((-0.08812918327148857 51.519410735327...,-0.092554,51.518200
2,3,E01000003,City of London 001C,City of London 001C,5.873326e+04,1170.012290,POLYGON ((-0.09455941138450838 51.522047047748...,-0.095832,51.521715
3,4,E01000004,City of London 001D,City of London 001D,2.292225e+06,9764.246314,POLYGON ((-0.07847165426305887 51.521509775013...,-0.093080,51.514237
4,5,E01000005,City of London 001E,City of London 001E,1.891322e+05,2202.802920,POLYGON ((-0.0773709615362531 51.5173959822039...,-0.075792,51.513510
5,6,E01000006,Barking and Dagenham 016A,Barking and Dagenham 016A,1.467746e+05,1907.214899,POLYGON ((0.09325749352991829 51.5378698381896...,0.088424,51.538971
6,7,E01000007,Barking and Dagenham 015A,Barking and Dagenham 015A,1.998638e+05,2864.770644,POLYGON ((0.07833531478168788 51.5423397919471...,0.077064,51.540438
7,8,E01000008,Barking and Dagenham 015B,Barking and Dagenham 015B,1.956970e+05,4055.213080,(POLYGON ((0.0690839438512655 51.5389052993088...,0.069647,51.540125
8,9,E01000009,Barking and Dagenham 016B,Barking and Dagenham 016B,1.280594e+05,2054.478108,"POLYGON ((0.08474649672928 51.5424942073335, 0...",0.083696,51.540488
9,10,E01000010,Barking and Dagenham 015C,Barking and Dagenham 015C,3.519925e+05,3191.239201,POLYGON ((0.08133280873358234 51.5393055104014...,0.078164,51.536911
